In [1]:
from pyspark.sql import SparkSession
from operator import add


# Spark session
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.218:7077") \
        .appName("Scalingtest")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.executor.memory","2g")\
        .getOrCreate()

# Spark context
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 19:43:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/09 19:43:08 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:
#loop to append file paths for each parent directory
root ='hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/'
letters = ['A','B','C','E','G','J','K','L','M','N','O','P','Q','R','S','T','U','W','X','Y','Z']
paths=[]
for i in letters:
    paths.append(root+i)
paths[-1]

'hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/Z'

In [5]:
#starting timing and run all cells below to measure wall time
import time
start = time.time()

#read in files from parent directories choose how may by slicing the paths list in the loop
data = spark_session.read.option("recursiveFileLookup", "true").json(paths[0])
for i in paths[1:3]:
    temp = spark_session.read.option("recursiveFileLookup", "true").json(i)
    data = data.union(temp)


#data = spark_session.read.json('hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/B/A/A/*.json')

In [10]:
# Flatten the tags column
from pyspark.sql.functions import flatten
dataframe = data.withColumn("New tags", flatten(data.tags))

In [11]:
# Remove the numbers from the tags (the numbers are every other element)
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf, col

removeNumbers = udf(lambda lst: lst[0::2], ArrayType(StringType()))

dataFrame = dataframe.withColumn("Only tags", removeNumbers(col("New tags")))

In [12]:
import re
# Function for determine if a song has been tagged with a tag including "male" or "female" or both
def genderTags(lst):
    genderList = []
    for element in lst:
        if re.search("female|Female", element) and "female" not in genderList:
            genderList.append("female")
            continue
        elif re.search("male|Male", element) and "male" not in genderList:
            genderList.append("male")
            continue
        
    return genderList

# Create udf from the above function
genderTagUDF = udf(genderTags, ArrayType(StringType()))

# Create a new column representing wether a song has been tagged with "female", "male" or both
dataframe = dataFrame.withColumn("Gender tag", genderTagUDF(col("Only tags")))
        

In [13]:
# Function for removing the gender tag from the tags
def removeGenderTags(lst):
    tags = []
    for element in lst:
        if re.search("female|Female", element) or re.search("male|Male", element):
            continue
        else:
            tags.append(element)
        
        
    return tags

# Create udf from the above function
removeGenderTagUDF = udf(removeGenderTags, ArrayType(StringType()))

# Create a new column where the"female" and "male" tags have been removed from the other tags
filteredDF = dataframe.withColumn("Tags", removeGenderTagUDF(col("Only tags")))

In [14]:
# Create new dataframe with only the tags and the gender tag
genderDataFrame = filteredDF.select("Tags", "Gender tag")

In [15]:
# Explode the tags column
from pyspark.sql.functions import explode
genderDataFrame = genderDataFrame.select(genderDataFrame["Gender tag"], explode(genderDataFrame["Tags"]))
genderDataFrame = genderDataFrame.withColumnRenamed("col", "Tag")

In [16]:
# Explode the gender tag column
genderDataFrame = genderDataFrame.select(explode(genderDataFrame["Gender tag"]), genderDataFrame["Tag"])
genderDataFrame = genderDataFrame.withColumnRenamed("col", "Gender")


In [17]:
# Find the most frequent tags for "female"
tagsFemale = genderDataFrame.select("Tag").filter(genderDataFrame["Gender"] == "female").groupBy("Tag").agg({"Tag": "count"})\
        .withColumnRenamed("count(Tag)","Frequency").orderBy("Frequency", ascending=False)

tagsFemale.show()

22/03/09 20:15:08 ERROR TaskSchedulerImpl: Lost executor 14 on 192.168.2.54: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:15:16 ERROR TaskSchedulerImpl: Lost executor 15 on 192.168.2.152: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:15:25 ERROR TaskSchedulerImpl: Lost executor 16 on 192.168.2.54: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:15:30 ERROR TaskSchedulerImpl: Lost executor 17 on 192.168.2.152: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:15:30 ERROR TaskSchedulerImpl: Ignoring update with state FINISHED for TID 64838 because its task set is gone (this is likely the resu

22/03/09 20:18:45 ERROR TaskSchedulerImpl: Lost executor 39 on 192.168.2.152: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:18:45 ERROR TaskSchedulerImpl: Ignoring update with state FINISHED for TID 67689 because its task set is gone (this is likely the result of receiving duplicate task finished status updates) or its executor has been marked as failed.
22/03/09 20:19:00 ERROR TaskSchedulerImpl: Lost executor 41 on 192.168.2.152: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22/03/09 20:19:00 ERROR TaskSchedulerImpl: Ignoring update with state FINISHED for TID 67894 because its task set is gone (this is likely the result of receiving duplicate task finished status updates) or its executor has been marked as failed.
22/03/09 20:19:01 ERROR TaskSchedulerImpl: Lost executor 40 on 192.168.2.54: Remot

KeyboardInterrupt: 

In [ ]:
# Find the most frequent tags for "male"
tagsMale = genderDataFrame.select("Tag").filter(genderDataFrame["Gender"] == "male").groupBy("Tag").agg({"Tag": "count"})\
        .withColumnRenamed("count(Tag)","Frequency").orderBy("Frequency", ascending=False)

print(tagsMale.show())
print()
print(f'execution time in minutes: {(time.time()-start)/60}')



In [18]:
spark_session.stop()